In [1]:
import importlib
import stockInfo
from stockInfo import create_and_compile_model
importlib.reload(stockInfo)
from stockInfo import fetch_stock_data, dataCollect
from tabulate import tabulate


tick = input('What is your Ticker Name?').upper()
start_date = input('Enter start date (YYYY-MM-DD):')
end_date = input('Enter end date (YYYY-MM-DD):')

print('Entered Ticker:', tick)

stock_result = fetch_stock_data(tick, start_date, end_date)

if stock_result['company_name'] != 'N/A' and stock_result['data']:
    print('Company Name:', stock_result['company_name'])
    print('Stock Data:')
    for col, values in stock_result['data'].items():
        print(f'{col}: {values}')
else:
    print('Error')


Entered Ticker: AAPL
[*********************100%%**********************]  1 of 1 completed
Company Name: Apple Inc.
Stock Data:
Open: [121.75, 120.9800033569336, 120.93000030517578, 119.02999877929688, 121.69000244140625, 122.54000091552734, 120.4000015258789, 121.41000366210938, 125.69999694824219, 124.05000305175781, 122.87999725341797, 119.9000015258789, 120.33000183105469, 123.33000183105469, 122.81999969482422, 119.54000091552734, 120.3499984741211, 121.6500015258789, 120.11000061035156, 121.6500015258789, 123.66000366210938, 123.87000274658203, 126.5, 125.83000183105469, 128.9499969482422, 129.8000030517578, 132.52000427246094, 132.44000244140625, 134.94000244140625, 133.82000732421875, 134.3000030517578, 133.50999450683594, 135.02000427246094, 132.36000061035156, 133.0399932861328, 132.16000366210938, 134.8300018310547, 135.00999450683594, 134.30999755859375, 136.47000122070312, 131.77999877929688, 132.0399932861328, 131.19000244140625, 129.1999969482422, 127.88999938964844, 130.

In [2]:
if stock_result is not None:
    print('Stock Data Tail for ' +stock_result['company_name']+':')
    for col, values in stock_result['data'].items():
        print(f'{col}: {values[-5:]}')
else:
    print('Error')

Stock Data Tail for Apple Inc.:
Open: [189.97999572753906, 190.2100067138672, 194.4499969482422, 193.6300048828125, 194.1999969482422]
High: [190.0500030517578, 194.39999389648438, 194.75999450683594, 195.0, 195.99000549316406]
Low: [187.4499969482422, 190.17999267578125, 192.11000061035156, 193.58999633789062, 193.6699981689453]
Close: [189.42999267578125, 193.4199981689453, 192.32000732421875, 194.27000427246094, 195.7100067138672]
Adj Close: [189.42999267578125, 193.4199981689453, 192.32000732421875, 194.27000427246094, 195.7100067138672]
Volume: [43389500, 66628400, 41089700, 47477700, 53377300]


In [3]:
import pandas as pd
if stock_result['company_name'] != 'N/A' and stock_result['data']:
    data_collect = dataCollect(
        ticker=tick,
        startDate = start_date,
        endDate = end_date
    )

    stock_data_dict = data_collect.compress_data_to_dict()

    if stock_data_dict is not None:
        stock_data_df = pd.DataFrame(stock_data_dict)
        print(stock_data_df.head())
        print(stock_data_df.tail())

[*********************100%%**********************]  1 of 1 completed
        Date       Close
0 2021-03-04  120.129997
1 2021-03-05  121.419998
2 2021-03-08  116.360001
3 2021-03-09  121.089996
4 2021-03-10  119.980003
          Date       Close
693 2023-12-04  189.429993
694 2023-12-05  193.419998
695 2023-12-06  192.320007
696 2023-12-07  194.270004
697 2023-12-08  195.710007


In [4]:
if stock_data_df is not None:
    model, rmse = create_and_compile_model(stock_data_df)
    print(model.summary())
    print('Root Mean Squared Error (RMSE): ', rmse)
    

Epoch 1/7
499/499 [==============================] - 31s 46ms/step - loss: 0.0105 - mean_absolute_error: 0.0799
Epoch 2/7
499/499 [==============================] - 20s 40ms/step - loss: 0.0041 - mean_absolute_error: 0.0496
Epoch 3/7
499/499 [==============================] - 21s 43ms/step - loss: 0.0033 - mean_absolute_error: 0.0442
Epoch 4/7
499/499 [==============================] - 20s 40ms/step - loss: 0.0027 - mean_absolute_error: 0.0405
Epoch 5/7
499/499 [==============================] - 21s 43ms/step - loss: 0.0024 - mean_absolute_error: 0.0386
Epoch 6/7
499/499 [==============================] - 23s 46ms/step - loss: 0.0023 - mean_absolute_error: 0.0369
Epoch 7/7
5/5 [==============================] - 3s 31ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                

In [5]:
import math
data = stock_data_df.filter(['Close'])
dataset = data.values
#Get the number of rows to train the model on
training_data_len = math.ceil(len(dataset) * .8)

training_data_len

559

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)